<a href="https://colab.research.google.com/github/foxtrotmike/CS909/blob/master/multilinear.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
import math
class Multilinear(nn.Module):
    def __init__(self, input_sizes, output_size):
        super(Multilinear, self).__init__()
        self.weight = nn.Parameter(torch.Tensor(*input_sizes, output_size))
        self.bias = nn.Parameter(torch.Tensor(output_size))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        if self.bias is not None:
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
            bound = 1 / math.sqrt(fan_in)
            nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, *inputs):
        result = torch.einsum('...i,...j,...k,ijkl->...l', *inputs, self.weight)
        if self.bias is not None:
            result += self.bias
        return result

# Example usage
x1 = torch.randn(10, 5)  # Batch size of 10, input size 5
x2 = torch.randn(10, 4)  # Batch size of 10, input size 4
x3 = torch.randn(10, 6)  # Batch size of 10, input size 6

layer = Multilinear((5, 4, 6), 7)  # Output size 7
output = layer(x1, x2, x3)


In [4]:
output.shape

torch.Size([10, 7])